In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [21]:
# Standard python libraries
import os
import requests

# Essential DS libraries
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, roc_curve, RocCurveDisplay, auc, precision_score, roc_auc_score, accuracy_score, f1_score
from sklearn.impute import SimpleImputer

import matplotlib.pyplot as plt
import seaborn as sns

# Experiments block
import wandb


pd.set_option('display.max_columns', None)

In [3]:
# Подготовим инфраструктуру для версионирования экспериментов
project_name = "Makarov_mfdp_unit_5"
exp_name = "exp_"
config = []

## Эксперимент 4 Стэкинг
Идея описана в соответствующей статье - https://alexanderdyakonov.wordpress.com/2017/03/10/c%D1%82%D0%B5%D0%BA%D0%B8%D0%BD%D0%B3-stacking-%D0%B8-%D0%B1%D0%BB%D0%B5%D0%BD%D0%B4%D0%B8%D0%BD%D0%B3-blending/

Мы загружаем N алгоритмов и по результатам их работы прогоняем метаалгоритм, выдающий решение задачи. Реализация на лучшем фича селекшене:
1. включая кросс валидацию и без неё (4.1)
2. последовательный прогон нескольких блэндингов (4.2)

Подробное описание реализации - https://github.com/Dyakonov/ml_hacks/blob/master/dj_stacking.ipynb

In [17]:
outliers_data_x = pd.read_csv("2_best_features_permutation.csv")
outliers_data_y = pd.read_csv("1_outliers_raw_data")["TARGET"]

In [6]:
# Возьмём пример стэкинга из статьи. Применять будем на отобранных фичах
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import cross_val_predict

class DjStacking(BaseEstimator, ClassifierMixin):  
    """Стэкинг моделей scikit-learn"""

    def __init__(self, models, ens_model):
        """
        Инициализация
        models - базовые модели для стекинга
        ens_model - мета-модель
        """
        self.models = models
        self.ens_model = ens_model
        self.n = len(models)
        self.valid = None
        
    def fit(self, X, y=None, p=0.25, cv=3, err=0.001, random_state=None):
        """
        Обучение стекинга
        p - в каком отношении делить на обучение / тест
            если p = 0 - используем всё обучение!
        cv  (при p=0) - сколько фолдов использовать
        err (при p=0) - величина случайной добавки к метапризнакам
        random_state - инициализация генератора
            
        """
        if (p > 0): # делим на обучение и тест
            # разбиение на обучение моделей и метамодели
            train, valid, y_train, y_valid = train_test_split(X, y, test_size=p, random_state=random_state)
            
            # заполнение матрицы для обучения метамодели
            self.valid = np.zeros((valid.shape[0], self.n))
            for t, clf in enumerate(self.models):
                clf.fit(train, y_train)
                self.valid[:, t] = clf.predict(valid)
                
            # обучение метамодели
            self.ens_model.fit(self.valid, y_valid)
            
        else: # используем всё обучение
            
            # для регуляризации - берём случайные добавки
            self.valid = err*np.random.randn(X.shape[0], self.n)
            
            for t, clf in enumerate(self.models):
                # это oob-ответы алгоритмов
                self.valid[:, t] += cross_val_predict(clf, X, y, cv=cv, n_jobs=-1, method='predict')
                # но сам алгоритм надо настроить
                clf.fit(X, y)
            
            # обучение метамодели
            self.ens_model.fit(self.valid, y)  
            

        return self
    


    def predict(self, X, y=None):
        """
        Работа стэкинга
        """
        # заполение матрицы для мета-классификатора
        X_meta = np.zeros((X.shape[0], self.n))
        
        for t, clf in enumerate(self.models):
            X_meta[:, t] = clf.predict(X)
        
        a = self.ens_model.predict(X_meta)
        
        return (a)

In [7]:
# Мы не можем отрисовать пространство в 32 фичи, потому оставим от функции лишь предсказание ROC_AUC

def run_and_plot(clf, X, y, label):
    a = clf.predict(X)
    print (label + ' AUC-ROC  = ' + str( roc_auc_score(y, a) ))

In [22]:
# Классификационные данные
X = outliers_data_x

imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
X_ = pd.DataFrame(imp_mean.fit_transform(X))
X_.columns = X.columns
X_ = X_.astype(X.dtypes.to_dict())

y = outliers_data_y

train_X, test_X, train_y, test_y = train_test_split(X_, y, train_size=0.8, random_state=42)

In [24]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

# Чтобы не попадать на ошибку lightGBM, нужно сменить имена переменных
import re
train_X = train_X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test_X = test_X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

knn1 = KNeighborsRegressor(n_neighbors=3)
knn1.fit(train_X, train_y)
run_and_plot(knn1, test_X, test_y, '3NN')

knn2 = KNeighborsRegressor(n_neighbors=10)
knn2.fit(train_X, train_y)
run_and_plot(knn2, test_X, test_y, '10NN')


rg0 = Ridge(alpha=0.01)
rg0.fit(train_X, train_y)
run_and_plot(rg0, test_X, test_y, 'ridge-0.01')

rg1 = Ridge(alpha=1.1)
rg1.fit(train_X, train_y)
run_and_plot(rg1, test_X, test_y, 'ridge-1.1')

rg2 = Ridge(alpha=100.1)
rg2.fit(train_X, train_y)
run_and_plot(rg2, test_X, test_y, 'ridge-100.1')


rf1 = RandomForestRegressor(n_estimators=100, max_depth=1)
rf1.fit(train_X, train_y)
run_and_plot(rf1, test_X, test_y, 'rf-d1')

rf2 = RandomForestRegressor(n_estimators=100, max_depth=5)
rf2.fit(train_X, train_y)
run_and_plot(rf2, test_X, test_y, 'rf-d5')

gbm1 = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.05, max_depth=2, n_estimators=200, objective='regression', class_weight='balanced')    
gbm1.fit(train_X, train_y)
run_and_plot(gbm1, test_X, test_y, 'gbm-d2')

gbm2 = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.05, max_depth=5, n_estimators=200, objective='regression', class_weight='balanced')    
gbm2.fit(train_X, train_y)
run_and_plot(gbm2, test_X, test_y, 'gbm-d5')

3NN AUC-ROC  = 0.5244997578692494
10NN AUC-ROC  = 0.5370755447941888
ridge-0.01 AUC-ROC  = 0.7551186440677966
ridge-1.1 AUC-ROC  = 0.7551351089588376
ridge-100.1 AUC-ROC  = 0.7520648910411623
rf-d1 AUC-ROC  = 0.6657554479418888
rf-d5 AUC-ROC  = 0.7783714285714285
gbm-d2 AUC-ROC  = 0.8455186440677966
gbm-d5 AUC-ROC  = 0.8486479418886197


In [25]:
# Запуск блэндинга и стеккинга
models = [knn1, knn2,rg1, rg2, rf1, rf2, gbm1, gbm2] # , rf3
ens_model = Ridge()
s1 = DjStacking(models, ens_model)
s1.fit(train_X, train_y)
run_and_plot(s1, test_X, test_y, '1-stacking')


s2 = DjStacking(models, ens_model)
s2.fit(train_X, train_y, p=-1)
run_and_plot(s1, test_X, test_y, '2-stacking')

1-stacking AUC-ROC  = 0.8452949152542373
2-stacking AUC-ROC  = 0.8431622276029055


In [26]:
# несколько блэндингов подряд
ens_model = Ridge(0.001)
s1 = DjStacking(models, ens_model)
a = 0
e = []
for t in range(10):
    s1.fit(train_X, train_y, p=0.4)
    a += s1.predict(test_X, train_y)
    
    auc = roc_auc_score(test_y, a)
    print (auc)
    e.append(auc)

0.8313249394673123
0.8400058111380145
0.8428590799031477
0.8439661016949154
0.8493220338983051
0.8505772397094431
0.8496900726392252
0.8487748184019371
0.8484164648910412
0.8485142857142858


In [27]:
# кросс валидация на разных фолдах

ens_model = Ridge(0.001)

s1 = DjStacking(models, ens_model)
a = 0
e = []
for t in range(2, 11):
    s1.fit(train_X, train_y, p=-1, cv=t, err=0.00)
    a = s1.predict(test_X, train_y)
    auc = roc_auc_score(test_y, a)
    print (auc)
    e.append(auc)

0.8446769975786925
0.8447292978208234
0.8413414043583536
0.8406886198547217
0.8391970944309927
0.8432154963680387
0.8425995157384989
0.8423467312348668
0.8424542372881356


In [28]:
arr1 = 1-s1.predict(test_X)
arr2 = s1.predict(test_X)
probas_for_roc_auc = np.stack((arr1, arr2), axis=1)
probas_for_roc_auc

array([[ 0.91957996,  0.08042004],
       [ 0.94826699,  0.05173301],
       [ 0.87873405,  0.12126595],
       ...,
       [ 0.79187181,  0.20812819],
       [ 1.00752206, -0.00752206],
       [ 0.95838414,  0.04161586]])

In [29]:
y_pred = s1.predict(test_X)

wandb.init(
        project=project_name, name=exp_name+"4"+"_stacking_regressors"
    )  # Инициализация эксперимента, project - название проекта
wandb.log(
    {
        "accuracy_score": accuracy_score(y_true=test_y, y_pred=y_pred.round()),
        "f1_score_weighted": f1_score(y_true=test_y, y_pred=y_pred.round(), average="weighted"),
        "precision_weighted": precision_score(y_true=test_y, y_pred=y_pred.round(), average="weighted"),
        "roc_auc_score": roc_auc_score(y_true=test_y, y_score=y_pred, average="weighted"),
        "gini": 2 * roc_auc_score(y_true=test_y, y_score=y_pred, average="weighted") - 1
    }
)
wandb.sklearn.plot_roc(y_true = test_y, y_probas = probas_for_roc_auc)
#wandb.sklearn.plot_class_proportions(y_train, y_test)
wandb.finish()

wandb: Currently logged in as: qeshtir (qesh-squad). Use `wandb login --relogin` to force relogin


wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.


accuracy_score,▁
f1_score_weighted,▁
gini,▁
precision_weighted,▁
roc_auc_score,▁
accuracy_score,0.94429
f1_score_weighted,0.91854
gini,0.68491
precision_weighted,0.9474
roc_auc_score,0.84245


### Предыдущий запуск был произведён на различного рода регрессорах. Попробуем собрать такой же стекинг, но на классификаторах

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier

In [31]:
estimators = [
    ('knn5', make_pipeline(StandardScaler(), 
                          KNeighborsClassifier(n_neighbors=5))),
    ('svc', make_pipeline(StandardScaler(), 
                          SVC(class_weight="balanced", 
                              probability=True, 
                              random_state=42))),
    ('rf', RandomForestClassifier(n_estimators=500, 
                                  max_depth=10, 
                                  min_samples_leaf=0.005, 
                                  class_weight="balanced", 
                                  random_state=42)),
    ('gbm', lgb.LGBMClassifier(boosting_type='gbdt', 
                                 learning_rate=0.05, 
                                 max_depth=10, 
                                 n_estimators=500,
                                 objective='binary', 
                                 class_weight='balanced',
                                 metric='auc',
                                 random_state=42)),
    ('mlp', MLPClassifier(hidden_layer_sizes=1000, 
                          alpha=0.1, 
                          random_state=42)),

]

clf = StackingClassifier(

    estimators=estimators, final_estimator=LogisticRegressionCV(class_weight="balanced"), cv=5, stack_method='predict_proba'

)

In [32]:
clf.fit(train_X, train_y).score(test_X, test_y)

0.8666666666666667

In [33]:
print(roc_auc_score(test_y, clf.predict(test_X)))
print(classification_report(test_y, clf.predict(test_X)))

0.7601937046004843
              precision    recall  f1-score   support

           0       0.98      0.88      0.93      4130
           1       0.24      0.64      0.35       250

    accuracy                           0.87      4380
   macro avg       0.61      0.76      0.64      4380
weighted avg       0.93      0.87      0.89      4380



In [34]:
y_pred = clf.predict(test_X)

wandb.init(
        project=project_name, name=exp_name+"4.1"+"_stacking_classifiers_mlp_knn"
    )  # Инициализация эксперимента, project - название проекта
wandb.log(
    {
        "accuracy_score": accuracy_score(y_true=test_y, y_pred=y_pred.round()),
        "f1_score_weighted": f1_score(y_true=test_y, y_pred=y_pred.round(), average="weighted"),
        "precision_weighted": precision_score(y_true=test_y, y_pred=y_pred.round(), average="weighted"),
        "roc_auc_score": roc_auc_score(y_true=test_y, y_score=y_pred, average="weighted"),
        "gini": 2 * roc_auc_score(y_true=test_y, y_score=y_pred, average="weighted") - 1
    }
)
wandb.sklearn.plot_roc(y_true = test_y, y_probas = clf.predict_proba(test_X))
#wandb.sklearn.plot_class_proportions(y_train, y_test)
wandb.finish()

accuracy_score,▁
f1_score_weighted,▁
gini,▁
precision_weighted,▁
roc_auc_score,▁
accuracy_score,0.86667
f1_score_weighted,0.89303
gini,0.52039
precision_weighted,0.93411
roc_auc_score,0.76019


### Выводы
Стэкинг также не показал значимого улучшения. Кроме того, сам алгоритм обладает довольно низкой интерпретируемостью. Гипотеза заключается в том, что мы выбрали сложный подход для достаточно простой зависимости, т.к. регрессоры справляются с задачей лучше.

Как лучший результат для тюнинга способ не подходит.
Стэкинг не обладает артефактами помимо массива предсказаний, выгрузить нечего.